# Food Spoilage Kernel Case Study

## Setup

In [1]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
import ipywidgets as widgets
from ipywidgets import interact, FloatSlider, IntSlider, Dropdown

from lifetime_modeling import *
from ablation_modeling import *

In [2]:
load_execution_time_from_csv("timing-ablation.csv")
load_embodied_values_from_csv("embodied-carbon-ablation.csv")
load_memory_from_csv("memory-ablation.csv")

## Lifetime Modeling

In [ ]:
workload_dropdown = widgets.Dropdown(
    options=execution_time.keys(),
    value="DT-Small",
    description="Kernel:",
)

lifetime_slider = widgets.FloatSlider(
    value=1.0,
    min=MIN_LIFETIME,
    max=MAX_LIFETIME,
    step=0.5,
    description='Lifetime (yrs)'
)

inf_freq_slider = widgets.BoundedFloatText(
    value=1.0,
    min=MIN_INF_FREQ,
    max=MAX_INF_FREQ,
    step=1.0,
    description='Number of Executed Tasks per Year'
)

power_source_dropdown = widgets.Dropdown(
    options=carbon_intensity_values.keys(),
    value=list(carbon_intensity_values.keys())[0],
    description="Power Source:",
)

carbon_intensity_box = widgets.Text(
    value=carbon_intensity_values[power_source_dropdown.value],
    description="Carbon Int. (CO₂/unit energy)",
)

carbon_intensity_box.disabled = True
def update_carbon_intensity(*args):
    if power_source_dropdown.value == "Custom":
        carbon_intensity_box.disabled = False
    else:
        carbon_intensity_box.value = carbon_intensity_values[power_source_dropdown.value]
        carbon_intensity_box.disabled = True

power_source_dropdown.observe(update_carbon_intensity, 'value')

core_freq_box = widgets.BoundedFloatText(
    value=10000.0,
    min=1000.0,
    max=100000.0,
    step=100.0,
    description='Core Freq'
)

ui = widgets.VBox([
    workload_dropdown,
    lifetime_slider,
    inf_freq_slider,
    power_source_dropdown,
    carbon_intensity_box,
    core_freq_box
])

out = widgets.interactive_output(
    plot_carbon_vs_lifetime,
    {
        'workload_choice': workload_dropdown,
        'lifetime_yrs': lifetime_slider,
        'inf_freq': inf_freq_slider,
        'carbon_intensity': carbon_intensity_box,
        'core_freq': core_freq_box,
    }
)

display(ui, out)

Output()

## Carbon-Accuracy Kernel Modeling

In [ ]:

import matplotlib.pyplot as plt

def plot_accuracy_vs_carbon_with_fig(lifetime_years, tasks_per_year, carbon_intensity):
    fig, ax = plt.subplots(figsize=(10, 6))
    plot_accuracy_vs_carbon(
        lifetime_years=lifetime_years,
        tasks_per_year=tasks_per_year,
        carbon_intensity=carbon_intensity,
        ax=ax
    )
    plt.show()

# Reasonable slider ranges for interactive exploration
interact(
    plot_accuracy_vs_carbon_with_fig,
    lifetime_years=FloatSlider(value=1, min=0.1, max=10, step=0.1, description='Lifetime (years)'),
    tasks_per_year=IntSlider(value=365, min=1, max=10000, step=1, description='Tasks/year'),
    carbon_intensity=Dropdown(
        options=carbon_intensity_values,
        value=carbon_intensity_values["US Avg (2023)"],
        description='Carbon Intensity\n(gCO2e/Wh)'
    )
)


interactive(children=(FloatSlider(value=1.0, description='Lifetime (years)', max=10.0, min=0.1), IntSlider(val…

<function __main__.plot_accuracy_vs_carbon_with_fig(lifetime_years, tasks_per_year, carbon_intensity)>

## At-scale Carbon Projections

In [5]:
compute_total_carbon("Serv", "LR", 1, 365, 367 / 3600 / 1000000, 10000)

(1.285745959, 0.001894388481443765, 1.2876403474814437)

In [6]:
ideal_carbon = 0.00129 + 0.00129 + 0.00828
conservative_carbon = 0.02 + 0.00129 + 0.107
silicon_carbon = 2.66

beef_kg_carbon = 14.5
beef_total_kg = 26.19 * 453.592 * 1_000_000
beef_total_carbon = beef_kg_carbon * beef_total_kg

beef_wasted_percent = 0.31

car_kg = 4600

config_carbons = {
    "Flexible": 0.01086,
    "Hybrid": 0.12829,
    "Silicon": 2.66
}

In [7]:
# Calculate savings for each variant at specified savings portions
savings_portions = [1.0, 0.1, 0.01, 0.001, 0.0001]  # 100%, 10%, 1%, 0.1%, 0.01%
savings_portion_labels = ["100%", "10%", "1%", "0.1%", "0.01%"]

variant_savings_results = {}
variant_cars_results = {}

for label, carbon_val in config_carbons.items():
    savings_list = []
    cars_list = []
    for portion in savings_portions:
        savings = beef_total_carbon * portion * beef_wasted_percent - beef_total_kg * carbon_val
        savings_list.append(savings)
        cars_equiv = savings / car_kg
        cars_list.append(cars_equiv)
    variant_savings_results[label] = savings_list
    variant_cars_results[label] = cars_list

import pandas as pd

# Define the desired row and column order
row_order = ["Ideal", "Conservative", "Silicon"]
col_order = savings_portion_labels

# Reorganize the DataFrame: rows = variants, columns = percentages
df_savings = pd.DataFrame(variant_savings_results, index=col_order).T

df_cars = pd.DataFrame(variant_cars_results, index=col_order).T

print(df_savings)
print(df_cars)



                  100%           10%            1%          0.1%         0.01%
Flexible  5.326968e+10  5.210857e+09  4.049747e+08 -7.561349e+07 -1.236723e+08
Hybrid    5.187466e+10  3.815838e+09 -9.900437e+08 -1.470632e+09 -1.518691e+09
Silicon   2.179902e+10 -2.625980e+10 -3.106568e+10 -3.154627e+10 -3.159433e+10
                  100%           10%            1%          0.1%         0.01%
Flexible  1.158036e+07  1.132795e+06  8.803798e+04 -1.643772e+04 -2.688528e+04
Hybrid    1.127710e+07  8.295300e+05 -2.152269e+05 -3.197026e+05 -3.301502e+05
Silicon   4.738917e+06 -5.708652e+06 -6.753409e+06 -6.857885e+06 -6.868332e+06


In [8]:
# For each variant, calculate the portion needed to achieve zero savings (i.e., set equation to zero and solve for portion)
required_portions = {}
for label, carbon_val in config_carbons.items():
    numerator = beef_total_kg * carbon_val
    denominator = beef_total_carbon * beef_wasted_percent
    if denominator != 0:
        portion = numerator / denominator
    else:
        portion = float('nan')
    required_portions[label] = portion

print("Portion required for each variant to reach zero savings (i.e., break-even point):")
for label, portion in required_portions.items():
    print(f"{label}: {portion:.6f}")


Portion required for each variant to reach zero savings (i.e., break-even point):
Flexible: 0.002416
Hybrid: 0.028541
Silicon: 0.591769
